In [25]:
import pandas as pd  # import the neccessary libraries
pd.set_option('display.max_columns', 300)  # display all the columns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsRegressor  # In case we want to use k-nearest neighbor model
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

plt.style.use('seaborn')
sns.set(style="white")

## Step 1: Read in hold out data, scalers, and best model

In [26]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [27]:
holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [28]:
import pickle 

pickle_in = open('model.pickle','rb')
final_model = pickle.load(pickle_in)

In [29]:
pickle_in = open('scaler.pickle','rb')
final_scaler = pickle.load(pickle_in)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values and scaling) that we performed on the original data.  

In [30]:
holdout['bed_bath'] = holdout['bedrooms'] + holdout['bathrooms']
holdout['renovated'] = holdout['yr_renovated'].apply(lambda x: (1 if  x > 0 else 0))
holdout['yrs_old'] = 2016 - holdout['yr_built']

In [31]:
zip_cat = pd.read_csv('zip_cat.csv', index_col = 0)
zip_cat

,price
zipcode,
98001,low
98002,low
98003,low
98004,very_high
98005,high
...,...
98177,high
98178,low
98188,low


In [32]:
holdout['zip_price_level'] = holdout['zipcode'].apply(lambda x: zip_cat.loc[x])
holdout['zip_price_level'].value_counts()

medium       1875
high         1377
low           881
very_high     190
Name: zip_price_level, dtype: int64

In [33]:
holdout = pd.get_dummies(holdout, columns = ['zip_price_level'], drop_first= True)
holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bed_bath,renovated,yrs_old,zip_price_level_low,zip_price_level_medium,zip_price_level_very_high
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,6.50,0,49,0,1,0
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,6.50,0,49,0,1,0
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,5.50,0,11,0,0,0
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,4.75,0,40,0,0,0
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,6.75,0,11,0,1,0


In [34]:
features =['sqft_living','floors','waterfront','view','grade','bed_bath','renovated','yrs_old',
          'sqft_lot15','sqft_living15','condition','zip_price_level_low','zip_price_level_medium',
           'zip_price_level_very_high']

In [35]:
holdout_df = holdout[features]
holdout_df.shape

(4323, 14)

In [36]:
holdout_df.head()

,sqft_living,floors,waterfront,view,grade,bed_bath,renovated,yrs_old,sqft_lot15,sqft_living15,condition,zip_price_level_low,zip_price_level_medium,zip_price_level_very_high
0,2270,1.0,0,0,8,6.50,0,49,10918,2020,3,0,1,0
1,2270,1.0,0,0,8,6.50,0,49,10918,2020,3,0,1,0
2,1470,2.0,0,0,8,5.50,0,11,1576,1470,3,0,0,0
3,1280,1.0,0,0,8,4.75,0,40,10565,1160,3,0,0,0
4,2830,2.0,0,0,8,6.75,0,11,7916,2830,3,0,1,0


In [37]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(holdout_df)
poly2_columns = poly_2.get_feature_names(holdout_df.columns)
holdout_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
print(holdout_poly2.shape)
holdout_poly2.head()

(4323, 119)


,sqft_living,floors,waterfront,view,grade,bed_bath,renovated,yrs_old,sqft_lot15,sqft_living15,condition,zip_price_level_low,zip_price_level_medium,zip_price_level_very_high,sqft_living^2,sqft_living floors,sqft_living waterfront,sqft_living view,sqft_living grade,sqft_living bed_bath,sqft_living renovated,sqft_living yrs_old,sqft_living sqft_lot15,sqft_living sqft_living15,sqft_living condition,sqft_living zip_price_level_low,sqft_living zip_price_level_medium,sqft_living zip_price_level_very_high,floors^2,floors waterfront,floors view,floors grade,floors bed_bath,floors renovated,floors yrs_old,floors sqft_lot15,floors sqft_living15,floors condition,floors zip_price_level_low,floors zip_price_level_medium,floors zip_price_level_very_high,waterfront^2,waterfront view,waterfront grade,waterfront bed_bath,waterfront renovated,waterfront yrs_old,waterfront sqft_lot15,waterfront sqft_living15,waterfront condition,waterfront zip_price_level_low,waterfront zip_price_level_medium,waterfront zip_price_level_very_high,view^2,view grade,view bed_bath,view renovated,view yrs_old,view sqft_lot15,view sqft_living15,view condition,view zip_price_level_low,view zip_price_level_medium,view zip_price_level_very_high,grade^2,grade bed_bath,grade renovated,grade yrs_old,grade sqft_lot15,grade sqft_living15,grade condition,grade zip_price_level_low,grade zip_price_level_medium,grade zip_price_level_very_high,bed_bath^2,bed_bath renovated,bed_bath yrs_old,bed_bath sqft_lot15,bed_bath sqft_living15,bed_bath condition,bed_bath zip_price_level_low,bed_bath zip_price_level_medium,bed_bath zip_price_level_very_high,renovated^2,renovated yrs_old,renovated sqft_lot15,renovated sqft_living15,renovated condition,renovated zip_price_level_low,renovated zip_price_level_medium,renovated zip_price_level_very_high,yrs_old^2,yrs_old sqft_lot15,yrs_old sqft_living15,yrs_old condition,yrs_old zip_price_level_low,yrs_old zip_price_level_medium,yrs_old zip_price_level_very_high,sqft_lot15^2,sqft_lot15 sqft_living15,sqft_lot15 condition,sqft_lot15 zip_price_level_low,sqft_lot15 zip_price_level_medium,sqft_lot15 zip_price_level_very_high,sqft_living15^2,sqft_living15 condition,sqft_living15 zip_price_level_low,sqft_living15 zip_price_level_medium,sqft_living15 zip_price_level_very_high,condition^2,condition zip_price_level_low,condition zip_price_level_medium,condition zip_price_level_very_high,zip_price_level_low^2,zip_price_level_low zip_price_level_medium,zip_price_level_low zip_price_level_very_high,zip_price_level_medium^2,zip_price_level_medium zip_price_level_very_high,zip_price_level_very_high^2
0,2270.0,1.0,0.0,0.0,8.0,6.50,0.0,49.0,10918.0,2020.0,3.0,0.0,1.0,0.0,5152900.0,2270.0,0.0,0.0,18160.0,14755.0,0.0,111230.0,24783860.0,4585400.0,6810.0,0.0,2270.0,0.0,1.0,0.0,0.0,8.0,6.50,0.0,49.0,10918.0,2020.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,52.0,0.0,392.0,87344.0,16160.0,24.0,0.0,8.0,0.0,42.2500,0.0,318.50,70967.00,13130.0,19.50,0.0,6.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2401.0,534982.0,98980.0,147.0,0.0,49.0,0.0,119202724.0,22054360.0,32754.0,0.0,10918.0,0.0,4080400.0,6060.0,0.0,2020.0,0.0,9.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2270.0,1.0,0.0,0.0,8.0,6.50,0.0,49.0,10918.0,2020.0,3.0,0.0,1.0,0.0,5152900.0,2270.0,0.0,0.0,18160.0,14755.0,0.0,111230.0,24783860.0,4585400.0,6810.0,0.0,2270.0,0.0,1.0,0.0,0.0,8.0,6.50,0.0,49.0,10918.0,2020.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,52.0,0.0,392.0,87344.0,16160.0,24.0,0.0,8.0,0.0,42.2500,0.0,318.50,70967.00,13130.0,19.50,0.0,6.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2401.0,534982.0,98980.0,147.0,0.0,49.0,0.0,119202724.0,22054360.0,32754.0,0.0,10918.0,0.0,4080400.0,6060.0,0.0,2020.0,0.0,9.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1470.0,2.0,0.0,0.0,8.0,5.50,0.0,11.0,1576.0,1470.0,3.0,0.0,0.0,0.0,2160900.0,2940.0,0.0,0.0,11760.0,8085.0,0.0,16170.0,2316720.0,2160900.0,4410.0

In [38]:
transformed_holdout = final_scaler.transform(holdout_poly2)

In [39]:
print(transformed_holdout.shape)
transformed_holdout

(4323, 119)


array([[ 0.20493158, -0.90984976, -0.08969788, ...,  1.16968746,
         0.        , -0.22083626],
       [ 0.20493158, -0.90984976, -0.08969788, ...,  1.16968746,
         0.        , -0.22083626],
       [-0.66464127,  0.94580372, -0.08969788, ..., -0.85492923,
         0.        , -0.22083626],
       ...,
       [-1.15377599,  0.94580372, -0.08969788, ..., -0.85492923,
         0.        , -0.22083626],
       [-0.52333568,  0.94580372, -0.08969788, ..., -0.85492923,
         0.        , -0.22083626],
       [-1.15377599,  0.94580372, -0.08969788, ..., -0.85492923,
         0.        , -0.22083626]])

## Step 3: Predict the holdout set

In [45]:
final_answers = final_model.predict(transformed_holdout)
df = pd.DataFrame(final_answers)

In [46]:
print(final_answers.shape)

(4323,)


In [47]:
df.head()

,0
0,507162.088201
1,507162.088201
2,500394.088201
3,461154.088201
4,508418.088201


## Step 4: Export your predictions

In [48]:
df.to_csv('housing_preds_Yasir_Alex.csv',header = None)